<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

## SmolAgents Orchestrator + Worker Tutorial

In this tutorial, we'll explore the orchestrator and worker approach using SmolAgents and how to trace the workflow with Phoenix.

The orchestrator and worker model is a powerful technique where a central orchestrator coordinates multiple worker agents, each responsible for specific tasks. This approach enhances the modularity and scalability of workflows. Tracing allows us to monitor this coordinated flow and understand the interactions between the orchestrator and workers.

SmolAgents provides flexible agent capabilities that integrate seamlessly with Python's asyncio library, enabling the orchestration of multiple agent interactions.

By the end of this tutorial, you'll learn how to:

- Set up SmolAgents with tools tailored for orchestrator and worker roles
- Implement the orchestrator and worker pattern using Python's asyncio library
- Utilize Phoenix to trace and visualize orchestrator and worker interactions
- Compare the performance of orchestrated versus non-orchestrated workflows

⚠️ You'll need a Hugging Face Token for this tutorial.

## Set up Keys and Dependencies


In [ ]:
!pip install openinference-instrumentation-smolagents smolagents

In [ ]:
import os
from getpass import getpass

from smolagents import CodeAgent, HfApiModel, tool

from phoenix.otel import register

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass("🔑 Enter your Hugging Face Token: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint")

## Configure Model

In [ ]:
# Initialize the model
model = HfApiModel()

## Configure Tracing


In [ ]:
tracer_provider = register(
    project_name="smolagents-agents",
    protocol="http/protobuf",
    auto_instrument=True,
)

## Define Tools

In [ ]:
@tool
def extract_qualifications(profile: str) -> str:
    """
    Extracts qualifications from a candidate's profile.

    Args:
        profile: Candidate background or resume summary.

    Returns:
        List of key skills, experiences, or education.
    """
    keywords = [word.strip(".,") for word in profile.split() if len(word) > 5]
    return ", ".join(set(keywords))


@tool
def analyze_tone(profile: str) -> str:
    """
    Analyzes tone or soft skills based on writing style.

    Args:
        profile: Candidate's self-description.

    Returns:
        Observations about communication style and tone.
    """
    return "Confident, technically focused, concise communicator."

## Worker Agents

In [ ]:
def resume_screener_agent(profile: str, role: str) -> str:
    agent = CodeAgent(tools=[extract_qualifications], model=model)
    prompt = (
        f"Review this profile for a role as {role}:\n\n{profile}\n\n"
        f"Use extract_qualifications to pull skills, then give a short score (1-10) based on relevance."
    )
    return agent.run(prompt)


def culture_fit_agent(profile: str) -> str:
    agent = CodeAgent(tools=[analyze_tone], model=model)
    prompt = (
        f"Based on the following candidate self-description:\n\n{profile}\n\n"
        f"Use analyze_tone to assess their communication style. Then comment on possible team or culture fit."
    )
    return agent.run(prompt)


def recommendation_agent(tech_eval: str, culture_eval: str) -> str:
    agent = CodeAgent(tools=[], model=model)
    prompt = (
        f"A recruiter received these evaluations:\n\n"
        f"Technical Evaluation:\n{tech_eval}\n\n"
        f"Culture Fit Evaluation:\n{culture_eval}\n\n"
        f"Based on both, should this candidate move to the next round? Give a yes/no and a short justification."
    )
    return agent.run(prompt)

## Orchestrator

In [ ]:
def orchestrator(candidate_name: str, profile: str, role: str):
    print(f"\n🧑‍💼 Reviewing: {candidate_name} — for role: {role}")

    # Step 1: Get tech screen
    tech_eval = resume_screener_agent(profile, role)

    # Step 2: Get culture screen
    culture_eval = culture_fit_agent(profile)

    # Step 3: Get final recommendation
    final_rec = recommendation_agent(tech_eval, culture_eval)

    # Show results
    print(f"\n📄 Technical Eval:\n{tech_eval}")
    print(f"\n💬 Culture Fit Eval:\n{culture_eval}")
    print(f"\n✅ Recommendation:\n{final_rec}")
    print("\n" + "=" * 60 + "\n")

## Run Multiple Candidates in Parallel 

In [ ]:
def main():
    role = "Machine Learning Engineer"
    candidates = {
        "Sofia Rao": """5+ years in applied ML. Experience with Transformers, model evaluation, and production pipelines. Previously led a small team at a healthtech startup.""",
        "Daniel Kim": """Worked as a backend engineer, now transitioning into ML. Strong Python and infra background. No formal ML experience, but completed multiple online LLM projects.""",
    }

    for name, profile in candidates.items():
        orchestrator(name, profile, role)


if __name__ == "__main__":
    main()